In [616]:
just_checking_integrity=False

In [617]:
import gc
import pandas as pd
import scipy.sparse as sps
import numpy as np
import tensorflow_ranking as tfr

In [618]:
URM_test=sps.load_npz("../dataset/processed_data/URM_test_final.npz")

In [619]:
URM_test.shape

(4439967, 23690)

In [620]:
np.ravel(URM_test.sum(axis=1)).shape

(4439967,)

In [621]:
Session_ids=np.argwhere(np.ravel(URM_test.sum(axis=1)))

In [622]:
is_sorted = lambda a: np.all(a[:-1] <= a[1:])

In [623]:
is_sorted(Session_ids)

True

In [624]:
# Create different datasets based on the selected list in order to compare between them
selected_list = ["UCF_W","ICBF_W","Graph_W","ICF_W"]

for model in selected_list:
    datasets = {}
    datasets_max = {}
#["UCF","ICBF","Graph","ICF","UCF_W","ICBF_W","Graph_W","ICF_W","UCF_WW","ICBF_WW","Graph_WW","ICF_WW"]

for model in selected_list:
    df_max = None
    df_feats = None
    print(gc.collect())
    print(f"loading {model}")
    df=pd.read_csv(f"../dataset/candidates/traditional_recs/final/{model}.csv")
    print(len(df))
    print(f"loaded {model}")
    for col in df:
        if "Max" in col:
            df_max=df[["Session_Id",col]].drop_duplicates()
        elif "Score" in col:
            df_feats=df[["Session_Id","Item_ID",col]]

    if df_feats is not None:
        datasets[model]=df_feats
    if df_max is not None:
        datasets_max[model]=df_max
    del df
        

#del dataset_max
del df_feats
del df_max

55
loading UCF_W
16350023
loaded UCF_W
0
loading ICBF_W
14855475
loaded ICBF_W
0
loading Graph_W
8321415
loaded Graph_W
0
loading ICF_W
17771428
loaded ICF_W


In [625]:
print(len(Session_ids))

50000


In [626]:
ids = {}
map_ids = {}
for model in selected_list:
    ids[model]=np.sort(datasets[model]["Session_Id"].unique())
    map_ids[model]=pd.DataFrame()
    map_ids[model]["Session_Id"]=ids[model]
    if just_checking_integrity:
        map_ids[model]["Real_Id"]=Session_ids[:len(ids[model])]
    else:
        map_ids[model]["Real_Id"]=Session_ids


In [627]:
for model in selected_list:
    datasets_max[model]=datasets_max[model].merge(map_ids[model],on="Session_Id",how="left")
    del datasets_max[model]["Session_Id"]
    datasets_max[model].rename(columns={"Real_Id":"Session_Id"},inplace=True)

In [628]:
for model in selected_list:
    datasets[model]=datasets[model].merge(map_ids[model],on="Session_Id",how="left")
    del datasets[model]["Session_Id"]
    datasets[model].rename(columns={"Real_Id":"Session_Id"},inplace=True)

In [629]:
import ast
submission_df_tr=pd.read_csv("../dataset/candidates/NN/Transformer/final/candidates.csv")
submission_df_tr["score"]=submission_df_tr["score"].map(ast.literal_eval)
submission_df_tr["code"]=submission_df_tr["code"].map(ast.literal_eval)
submission_df_tr["rank"]=submission_df_tr["rank"].map(ast.literal_eval)
submission_df_tr = submission_df_tr.explode(['code','score', 'rank'])
submission_df_tr[["session_id","code","rank"]]=submission_df_tr[["session_id","code","rank"]].astype(int)
submission_df_tr.rename(columns={"code":"Item_ID","session_id":"Session_Id","rank":"rank_transformer","score":"score_transformer"},inplace=True)

In [630]:
for model in selected_list:
    datasets[model]=datasets[model].merge(submission_df_tr,on=["Session_Id","Item_ID"],how="outer")

del submission_df_tr


In [631]:
print(datasets["UCF_W"].shape)
print(datasets_max["UCF_W"].shape)

(18529364, 5)
(50000, 2)


In [632]:
datasets["UCF_W"].head()

,Item_ID,Score_UCF_W,Session_Id,score_transformer,rank_transformer
0,11,0.034741,61,NaN,NaN
1,19,0.017621,61,NaN,NaN
2,23,0.040097,61,NaN,NaN
3,25,0.024129,61,NaN,NaN
4,69,0.053659,61,0.009634,9.0


In [633]:
datasets_max["UCF_W"].head()

,Max_Score_UCF_W,Session_Id
0,206.496190,61
1,586.109500,96
2,571.725100,185
3,64.075720,224
4,18.130371,285


In [634]:
for model in selected_list:
    print(f"Working on {model}")
    datasets[model]=datasets[model].merge(datasets_max[model],on=["Session_Id"],how="left")    
    del datasets_max[model]

Working on UCF_W
Working on ICBF_W
Working on Graph_W
Working on ICF_W


In [635]:
for model in selected_list:
    datasets[model].fillna(0,inplace=True)

/var/folders/yt/h_zrk_k931s4wj_999xrfy0m0000gq/T/ipykernel_17928/4016355296.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  datasets[model].fillna(0,inplace=True)


In [636]:
gc.collect()

2508

In [637]:
dataset_macro_features = pd.read_csv('../dataset/processed_data/macro_feats_NN_final.csv',
                                     usecols=['session_id',
                                                'session_similarity',
                                                'session_similarity_uniques', 'session_similarity_feats', 'session_similarity_feats_uniques',
                                              'length_of_session_seconds',
                   'avg_time_spent_per_item_seconds',
                   'variance_time_spent_per_item_seconds',
                   'n_seen_items',
                   'n_unique_items',])

dataset_macro_features.rename(columns={'session_id':'Session_Id'}, inplace=True)
for model in selected_list:
    datasets[model] = datasets[model].merge(dataset_macro_features, on='Session_Id', how='left')

In [638]:
# dataset.drop('Max_Score_TopPop15', axis=1, inplace=True)

In [639]:
datasets["UCF_W"].head()

,Item_ID,Score_UCF_W,Session_Id,score_transformer,rank_transformer,Max_Score_UCF_W,length_of_session_seconds,avg_time_spent_per_item_seconds,variance_time_spent_per_item_seconds,n_seen_items,n_unique_items,session_similarity,session_similarity_uniques,session_similarity_feats,session_similarity_feats_uniques
0,11,0.034741,61,0.000000,0.0,206.49619,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
1,19,0.017621,61,0.000000,0.0,206.49619,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
2,23,0.040097,61,0.000000,0.0,206.49619,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
3,25,0.024129,61,0.000000,0.0,206.49619,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
4,69,0.053659,61,0.009634,9.0,206.49619,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0


In [640]:
datasets["ICBF_W"].head()

,Item_ID,Score_ICBF_W,Session_Id,score_transformer,rank_transformer,Max_Score_ICBF_W,length_of_session_seconds,avg_time_spent_per_item_seconds,variance_time_spent_per_item_seconds,n_seen_items,n_unique_items,session_similarity,session_similarity_uniques,session_similarity_feats,session_similarity_feats_uniques
0,11,0.115033,61,0.000000,0.0,3.357151,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
1,23,0.072503,61,0.000000,0.0,3.357151,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
2,25,0.081094,61,0.000000,0.0,3.357151,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
3,31,0.081091,61,0.000000,0.0,3.357151,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
4,69,0.085104,61,0.009634,9.0,3.357151,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0


In [641]:
datasets["ICF_W"].head()

,Item_ID,Score_ICF_W,Session_Id,score_transformer,rank_transformer,Max_Score_ICF_W,length_of_session_seconds,avg_time_spent_per_item_seconds,variance_time_spent_per_item_seconds,n_seen_items,n_unique_items,session_similarity,session_similarity_uniques,session_similarity_feats,session_similarity_feats_uniques
0,11,0.311263,61,0.000000,0.0,0.563566,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
1,23,0.325638,61,0.000000,0.0,0.563566,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
2,25,0.006163,61,0.000000,0.0,0.563566,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
3,69,0.338065,61,0.009634,9.0,0.563566,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
4,78,0.272552,61,0.001693,74.0,0.563566,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0


In [642]:
datasets["Graph_W"].head()

,Item_ID,Score_Graph_W,Session_Id,score_transformer,rank_transformer,Max_Score_Graph_W,length_of_session_seconds,avg_time_spent_per_item_seconds,variance_time_spent_per_item_seconds,n_seen_items,n_unique_items,session_similarity,session_similarity_uniques,session_similarity_feats,session_similarity_feats_uniques
0,11,0.006554,61,0.000000,0.0,12.915223,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
1,23,0.007389,61,0.000000,0.0,12.915223,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
2,25,0.004554,61,0.000000,0.0,12.915223,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
3,69,0.011284,61,0.009634,9.0,12.915223,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
4,78,0.006321,61,0.001693,74.0,12.915223,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0


In [643]:
gc.collect()

0

In [644]:
for model in selected_list:
    datasets[model]=datasets[model].sort_values("Session_Id")

In [645]:
gc.collect()

0

In [646]:
display(datasets["UCF_W"])

,Item_ID,Score_UCF_W,Session_Id,score_transformer,rank_transformer,Max_Score_UCF_W,length_of_session_seconds,avg_time_spent_per_item_seconds,variance_time_spent_per_item_seconds,n_seen_items,n_unique_items,session_similarity,session_similarity_uniques,session_similarity_feats,session_similarity_feats_uniques
0,11,0.034741,61,0.000000,0.0,206.49619,0.0,0.0,0.0,1.0,1.0,1.000000,1.000000,1.000000,1.000000
250,3374,0.000000,61,0.000000,0.0,206.49619,0.0,0.0,0.0,1.0,1.0,1.000000,1.000000,1.000000,1.000000
249,3373,0.000000,61,0.000000,0.0,206.49619,0.0,0.0,0.0,1.0,1.0,1.000000,1.000000,1.000000,1.000000
248,3372,0.004067,61,0.000000,0.0,206.49619,0.0,0.0,0.0,1.0,1.0,1.000000,1.000000,1.000000,1.000000
247,3371,0.000000,61,0.000000,0.0,206.49619,0.0,0.0,0.0,1.0,1.0,1.000000,1.000000,1.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18529061,2301,0.177244,4439966,0.000000,0.0,27.67512,101.0,101.0,0.0,2.0,2.0,0.612814,0.612814,0.680839,0.680839
18529060,2299,0.130546,4439966,0.000000,0.0,27.67512,101.0,101.0,0.0,2.0,2.0,0.612814,0.612814,0.680839,0.680839
18529059,2247,0.000000,4439966,0.001897,59.0,27.67512,101.0,101.0,0.0,2.0,2.0,0.612814,0.612814,0.680839,0.680839
18529070,2514,0.000000,4439966,0.004932,24.0,27.67512,101.0,101.0,0.0,2.0,2.0,0.612814,0.612814,0.680839,0.680839


In [647]:
display(datasets["ICBF_W"])

,Item_ID,Score_ICBF_W,Session_Id,score_transformer,rank_transformer,Max_Score_ICBF_W,length_of_session_seconds,avg_time_spent_per_item_seconds,variance_time_spent_per_item_seconds,n_seen_items,n_unique_items,session_similarity,session_similarity_uniques,session_similarity_feats,session_similarity_feats_uniques
0,11,0.115033,61,0.000000,0.0,3.357151,0.0,0.0,0.0,1.0,1.0,1.000000,1.000000,1.000000,1.000000
284,3385,0.000000,61,0.000000,0.0,3.357151,0.0,0.0,0.0,1.0,1.0,1.000000,1.000000,1.000000,1.000000
283,3384,0.000000,61,0.000000,0.0,3.357151,0.0,0.0,0.0,1.0,1.0,1.000000,1.000000,1.000000,1.000000
282,3383,0.000000,61,0.000000,0.0,3.357151,0.0,0.0,0.0,1.0,1.0,1.000000,1.000000,1.000000,1.000000
281,3382,0.000000,61,0.000000,0.0,3.357151,0.0,0.0,0.0,1.0,1.0,1.000000,1.000000,1.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18097743,2328,0.613297,4439966,0.000000,0.0,4.047759,101.0,101.0,0.0,2.0,2.0,0.612814,0.612814,0.680839,0.680839
18097742,2247,0.613297,4439966,0.001897,59.0,4.047759,101.0,101.0,0.0,2.0,2.0,0.612814,0.612814,0.680839,0.680839
18097741,2246,0.000000,4439966,0.004261,30.0,4.047759,101.0,101.0,0.0,2.0,2.0,0.612814,0.612814,0.680839,0.680839
18097751,2542,0.000000,4439966,0.001194,98.0,4.047759,101.0,101.0,0.0,2.0,2.0,0.612814,0.612814,0.680839,0.680839


In [648]:
display(datasets["ICF_W"])

,Item_ID,Score_ICF_W,Session_Id,score_transformer,rank_transformer,Max_Score_ICF_W,length_of_session_seconds,avg_time_spent_per_item_seconds,variance_time_spent_per_item_seconds,n_seen_items,n_unique_items,session_similarity,session_similarity_uniques,session_similarity_feats,session_similarity_feats_uniques
0,11,0.311263,61,0.000000,0.0,0.563566,0.0,0.0,0.0,1.0,1.0,1.000000,1.000000,1.000000,1.000000
264,3378,0.000000,61,0.000000,0.0,0.563566,0.0,0.0,0.0,1.0,1.0,1.000000,1.000000,1.000000,1.000000
263,3377,0.000000,61,0.000000,0.0,0.563566,0.0,0.0,0.0,1.0,1.0,1.000000,1.000000,1.000000,1.000000
262,3374,0.000000,61,0.000000,0.0,0.563566,0.0,0.0,0.0,1.0,1.0,1.000000,1.000000,1.000000,1.000000
261,3373,0.000000,61,0.000000,0.0,0.563566,0.0,0.0,0.0,1.0,1.0,1.000000,1.000000,1.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20068735,2047,0.000000,4439966,0.001623,67.0,0.432356,101.0,101.0,0.0,2.0,2.0,0.612814,0.612814,0.680839,0.680839
20068736,2050,0.019327,4439966,0.000000,0.0,0.432356,101.0,101.0,0.0,2.0,2.0,0.612814,0.612814,0.680839,0.680839
20068737,2054,0.453510,4439966,0.000000,0.0,0.432356,101.0,101.0,0.0,2.0,2.0,0.612814,0.612814,0.680839,0.680839
20068731,2017,0.000000,4439966,0.005671,21.0,0.432356,101.0,101.0,0.0,2.0,2.0,0.612814,0.612814,0.680839,0.680839


In [649]:
display(datasets["Graph_W"])

,Item_ID,Score_Graph_W,Session_Id,score_transformer,rank_transformer,Max_Score_Graph_W,length_of_session_seconds,avg_time_spent_per_item_seconds,variance_time_spent_per_item_seconds,n_seen_items,n_unique_items,session_similarity,session_similarity_uniques,session_similarity_feats,session_similarity_feats_uniques
0,11,0.006554,61,0.000000,0.0,12.915223,0.0,0.0,0.0,1.0,1.0,1.000000,1.000000,1.000000,1.000000
127,3172,0.005235,61,0.000000,0.0,12.915223,0.0,0.0,0.0,1.0,1.0,1.000000,1.000000,1.000000,1.000000
128,3195,0.000000,61,0.007192,16.0,12.915223,0.0,0.0,0.0,1.0,1.0,1.000000,1.000000,1.000000,1.000000
129,3197,0.009881,61,0.003530,41.0,12.915223,0.0,0.0,0.0,1.0,1.0,1.000000,1.000000,1.000000,1.000000
130,3203,0.000000,61,0.002134,58.0,12.915223,0.0,0.0,0.0,1.0,1.0,1.000000,1.000000,1.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10819600,1757,0.000000,4439966,0.001259,92.0,0.587271,101.0,101.0,0.0,2.0,2.0,0.612814,0.612814,0.680839,0.680839
10819601,1765,0.000000,4439966,0.002866,41.0,0.587271,101.0,101.0,0.0,2.0,2.0,0.612814,0.612814,0.680839,0.680839
10819602,1790,0.000000,4439966,0.001936,57.0,0.587271,101.0,101.0,0.0,2.0,2.0,0.612814,0.612814,0.680839,0.680839
10819604,1806,0.000000,4439966,0.001185,100.0,0.587271,101.0,101.0,0.0,2.0,2.0,0.612814,0.612814,0.680839,0.680839


In [650]:
import lightgbm

In [651]:
k=10
result = {}

In [652]:
for model_name in selected_list:
    result[model_name] = []
    for index in range(k):
        print(f"Current iteration: {str(index + 1)} of {str(k)} in model {model_name}")
        model = lightgbm.Booster(model_file=f'../rankers/model_{model_name}_{index}.txt')
        result[model_name].append(model.predict(datasets[model_name].drop(["Session_Id","Item_ID"],axis=1)))
    
    result[model_name] = np.array(result[model_name])
    result[model_name]

Current iteration: 1 of 10 in model UCF_W
Current iteration: 2 of 10 in model UCF_W
Current iteration: 3 of 10 in model UCF_W
Current iteration: 4 of 10 in model UCF_W
Current iteration: 5 of 10 in model UCF_W
Current iteration: 6 of 10 in model UCF_W
Current iteration: 7 of 10 in model UCF_W
Current iteration: 8 of 10 in model UCF_W
Current iteration: 9 of 10 in model UCF_W
Current iteration: 10 of 10 in model UCF_W
Current iteration: 1 of 10 in model ICBF_W
Current iteration: 2 of 10 in model ICBF_W
Current iteration: 3 of 10 in model ICBF_W
Current iteration: 4 of 10 in model ICBF_W
Current iteration: 5 of 10 in model ICBF_W
Current iteration: 6 of 10 in model ICBF_W
Current iteration: 7 of 10 in model ICBF_W
Current iteration: 8 of 10 in model ICBF_W
Current iteration: 9 of 10 in model ICBF_W
Current iteration: 10 of 10 in model ICBF_W
Current iteration: 1 of 10 in model Graph_W
Current iteration: 2 of 10 in model Graph_W
Current iteration: 3 of 10 in model Graph_W
Current iteratio

In [653]:
for model_name in selected_list:
    datasets[model_name]["score"]=np.mean(result[model_name] ,axis=0)
    # dataset["score"]=np.median(result ,axis=0)

In [654]:
datasets_final = {}
for model_name in selected_list:
    datasets_final[model_name]=datasets[model_name][["Session_Id","Item_ID","score"]]
    gc.collect()

In [655]:
datasets_final["UCF_W"].head()

,Session_Id,Item_ID,score
0,61,11,-1.606178
250,61,3374,-2.353994
249,61,3373,-2.353994
248,61,3372,-2.353994
247,61,3371,-2.353994


In [656]:
datasets_final["ICBF_W"].head()

,Session_Id,Item_ID,score
0,61,11,-1.522799
284,61,3385,-2.326241
283,61,3384,-2.326241
282,61,3383,-2.326241
281,61,3382,-2.326241


In [657]:
datasets_final["ICF_W"].head()

,Session_Id,Item_ID,score
0,61,11,-1.311609
264,61,3378,-2.959374
263,61,3377,-2.959374
262,61,3374,-2.959374
261,61,3373,-2.959374


In [658]:
datasets_final["Graph_W"].head()

,Session_Id,Item_ID,score
0,61,11,-1.785555
127,61,3172,-1.840658
128,61,3195,-1.371817
129,61,3197,-0.810337
130,61,3203,-1.841526


In [659]:
for model_name in selected_list:
    datasets_final[model_name]=datasets_final[model_name].groupby(['Session_Id']).agg(list).reset_index()

In [660]:
def get_indices(x):
    x=np.array(x)
    indices_unsorted=np.argpartition(x, -100)[-100:]
    x=x[indices_unsorted]
    indices=indices_unsorted[np.argsort(-x)]
    return indices

In [661]:
for model_name in selected_list:
    datasets_final[model_name]["indices"]=datasets_final[model_name]["score"].apply(lambda x: get_indices(x))

In [662]:
datasets_final["UCF_W"].iloc[0]

Session_Id                                                   61
Item_ID       [11, 3374, 3373, 3372, 3371, 3370, 3369, 3368,...
score         [-1.606178025909206, -2.3539937031097713, -2.3...
indices       [230, 174, 133, 187, 236, 243, 310, 266, 148, ...
Name: 0, dtype: object

In [663]:
datasets_final["ICBF_W"].iloc[0]

Session_Id                                                   61
Item_ID       [11, 3385, 3384, 3383, 3382, 3381, 3380, 3373,...
score         [-1.5227989811296556, -2.32624080370014, -2.32...
indices       [225, 240, 115, 161, 273, 264, 369, 313, 299, ...
Name: 0, dtype: object

In [664]:
datasets_final["ICF_W"].iloc[0]

Session_Id                                                   61
Item_ID       [11, 3378, 3377, 3374, 3373, 3372, 3371, 3370,...
score         [-1.3116089658295265, -2.9593744812377207, -2....
indices       [206, 229, 100, 189, 258, 247, 261, 106, 274, ...
Name: 0, dtype: object

In [665]:
datasets_final["Graph_W"].iloc[0]

Session_Id                                                   61
Item_ID       [11, 3172, 3195, 3197, 3203, 3251, 3288, 3290,...
score         [-1.7855548716949738, -1.8406581996407134, -1....
indices       [114, 23, 53, 188, 139, 132, 44, 122, 12, 5, 7...
Name: 0, dtype: object

In [666]:
datasets_final["UCF_W"].iloc[0]["Item_ID"][236]

292

In [667]:
datasets_final["ICBF_W"].iloc[0]["Item_ID"][236]

1635

In [668]:
datasets_final["ICF_W"].iloc[0]["Item_ID"][236]

1316

In [669]:
datasets_final["Graph_W"].iloc[0]["Item_ID"][136]

241

In [670]:
def get_items(x):
    items=np.array(x[0])
    return items[x[1]]

In [671]:
for model_name in selected_list:
    datasets_final[model_name]["predictions"]=datasets_final[model_name][["Item_ID","indices"]].apply(get_items,axis=1)

/var/folders/yt/h_zrk_k931s4wj_999xrfy0m0000gq/T/ipykernel_17928/744460501.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  items=np.array(x[0])
/var/folders/yt/h_zrk_k931s4wj_999xrfy0m0000gq/T/ipykernel_17928/744460501.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return items[x[1]]
/var/folders/yt/h_zrk_k931s4wj_999xrfy0m0000gq/T/ipykernel_17928/744460501.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  items=np.array(

In [672]:
for model_name in selected_list:
    datasets_final[model_name]=datasets_final[model_name][["Session_Id","predictions"]]
    datasets_final[model_name]["rank"]=datasets_final[model_name].apply(lambda x:tuple([rank for rank in range(1, 101)]) ,axis=1)

In [673]:
datasets_final["UCF_W"]

,Session_Id,predictions,rank
0,61,"[1045, 3586, 4574, 1563, 292, 143, 2995, 699, ...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
1,96,"[3564, 2328, 3778, 2346, 4097, 1356, 4587, 411...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
2,185,"[4146, 4148, 1936, 2330, 1652, 3326, 2704, 416...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
3,224,"[3338, 4202, 1437, 2769, 16, 2785, 1516, 4990,...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
4,285,"[3691, 3555, 2970, 3441, 3599, 465, 2268, 2471...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
...,...,...,...
49995,4439646,"[270, 1468, 191, 3618, 3656, 3037, 707, 1827, ...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
49996,4439648,"[4145, 3901, 3223, 2491, 4586, 1605, 3550, 535...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
49997,4439675,"[4865, 3116, 4741, 4985, 320, 2780, 3879, 1048...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
49998,4439868,"[519, 292, 2285, 1972, 3586, 3414, 4740, 750, ...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."


In [674]:
datasets_final["ICBF_W"]

,Session_Id,predictions,rank
0,61,"[1045, 1563, 4574, 3586, 143, 292, 2156, 69, 6...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
1,96,"[4097, 3564, 737, 3303, 3778, 957, 2499, 2328,...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
2,185,"[4146, 4148, 2330, 1936, 1652, 2704, 2643, 322...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
3,224,"[1437, 3338, 4202, 3026, 1245, 3343, 1899, 276...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
4,285,"[3441, 3555, 2970, 3691, 2268, 2899, 1070, 488...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
...,...,...,...
49995,4439646,"[270, 191, 707, 3037, 3618, 3656, 4370, 4913, ...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
49996,4439648,"[4145, 3901, 2234, 3223, 4586, 1636, 991, 1605...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
49997,4439675,"[779, 3116, 3247, 1906, 3398, 1169, 603, 819, ...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
49998,4439868,"[4740, 519, 2285, 292, 23, 750, 3414, 4220, 47...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."


In [675]:
datasets_final["ICF_W"]

,Session_Id,predictions,rank
0,61,"[1045, 1563, 4574, 3586, 143, 292, 69, 4436, 6...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
1,96,"[3564, 3778, 4097, 2346, 4756, 3607, 1608, 158...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
2,185,"[4146, 4148, 2330, 1936, 4162, 2704, 1652, 382...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
3,224,"[3338, 1437, 4202, 2769, 16, 881, 2785, 4990, ...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
4,285,"[3691, 3555, 2970, 3441, 2268, 1070, 2899, 488...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
...,...,...,...
49995,4439646,"[270, 191, 3618, 707, 4749, 3633, 2720, 3656, ...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
49996,4439648,"[4145, 3901, 2491, 3223, 4586, 3110, 3550, 223...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
49997,4439675,"[3166, 1906, 3247, 3116, 3398, 779, 3831, 4966...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
49998,4439868,"[519, 1972, 292, 2285, 3414, 4740, 750, 4220, ...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."


In [676]:
datasets_final["Graph_W"]

,Session_Id,predictions,rank
0,61,"[1045, 3586, 4574, 1563, 292, 143, 2995, 699, ...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
1,96,"[3564, 3778, 4097, 2328, 2346, 1356, 4119, 345...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
2,185,"[4146, 4148, 2330, 1936, 1652, 3326, 2704, 416...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
3,224,"[3338, 4202, 1437, 2769, 16, 881, 4990, 2180, ...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
4,285,"[3691, 2970, 3555, 3441, 2471, 3599, 2268, 465...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
...,...,...,...
49995,4439646,"[270, 1468, 3618, 191, 3037, 3656, 707, 1827, ...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
49996,4439648,"[4145, 3901, 3223, 2491, 1605, 3203, 535, 2234...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
49997,4439675,"[4865, 3116, 4985, 3879, 779, 3247, 819, 4966,...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
49998,4439868,"[519, 292, 2285, 1972, 3414, 3586, 4740, 750, ...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."


In [677]:
for model_name in selected_list:
    datasets_final[model_name]["predictions"]=datasets_final[model_name]["predictions"].apply(tuple)

In [678]:
for model_name in selected_list:
    datasets_final[model_name]=datasets_final[model_name][["Session_Id","predictions","rank"]]

In [679]:
for model_name in selected_list:
    datasets_final[model_name].columns=["session_id","code","rank"]

In [680]:
datasets_final["UCF_W"].head()

,session_id,code,rank
0,61,"(1045, 3586, 4574, 1563, 292, 143, 2995, 699, ...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
1,96,"(3564, 2328, 3778, 2346, 4097, 1356, 4587, 411...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
2,185,"(4146, 4148, 1936, 2330, 1652, 3326, 2704, 416...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
3,224,"(3338, 4202, 1437, 2769, 16, 2785, 1516, 4990,...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
4,285,"(3691, 3555, 2970, 3441, 3599, 465, 2268, 2471...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."


In [681]:
datasets_final["ICBF_W"].head()

,session_id,code,rank
0,61,"(1045, 1563, 4574, 3586, 143, 292, 2156, 69, 6...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
1,96,"(4097, 3564, 737, 3303, 3778, 957, 2499, 2328,...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
2,185,"(4146, 4148, 2330, 1936, 1652, 2704, 2643, 322...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
3,224,"(1437, 3338, 4202, 3026, 1245, 3343, 1899, 276...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
4,285,"(3441, 3555, 2970, 3691, 2268, 2899, 1070, 488...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."


In [682]:
datasets_final["ICF_W"].head()

,session_id,code,rank
0,61,"(1045, 1563, 4574, 3586, 143, 292, 69, 4436, 6...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
1,96,"(3564, 3778, 4097, 2346, 4756, 3607, 1608, 158...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
2,185,"(4146, 4148, 2330, 1936, 4162, 2704, 1652, 382...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
3,224,"(3338, 1437, 4202, 2769, 16, 881, 2785, 4990, ...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
4,285,"(3691, 3555, 2970, 3441, 2268, 1070, 2899, 488...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."


In [683]:
datasets_final["Graph_W"].head()

,session_id,code,rank
0,61,"(1045, 3586, 4574, 1563, 292, 143, 2995, 699, ...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
1,96,"(3564, 3778, 4097, 2328, 2346, 1356, 4119, 345...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
2,185,"(4146, 4148, 2330, 1936, 1652, 3326, 2704, 416...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
3,224,"(3338, 4202, 1437, 2769, 16, 881, 4990, 2180, ...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
4,285,"(3691, 2970, 3555, 3441, 2471, 3599, 2268, 465...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."


In [684]:
submissions_df = {}
for model_name in selected_list:
    submissions_df[model_name] = datasets_final[model_name].explode(['code', 'rank']).astype(int)

In [685]:
mapping= pd.read_csv('../dataset/processed_data/map.csv')
for model_name in selected_list:
    submissions_df[model_name]=submissions_df[model_name].merge(mapping,on="code",how="left")
    del submissions_df[model_name]["code"]
    submissions_df[model_name]=submissions_df[model_name][["session_id","item_id","rank"]]

In [686]:
print(len(submissions_df["UCF_W"]))

5000000


In [687]:
submissions_df["UCF_W"].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000000 entries, 0 to 4999999
Data columns (total 3 columns):
 #   Column      Dtype
---  ------      -----
 0   session_id  int64
 1   item_id     int64
 2   rank        int64
dtypes: int64(3)
memory usage: 114.4 MB


In [688]:
submissions_df["UCF_W"].isna().sum()

session_id    0
item_id       0
rank          0
dtype: int64

In [689]:
for model_name in selected_list:
    submissions_df[model_name].to_csv(
        f'../final_sub_{model_name}.csv',
        index=False
        )

In [690]:
test_sessions= pd.read_csv("../dataset/original_data/test_final_purchases.csv")
#test_sessions_full=pd.read_csv("../dataset/processed_data/test_final_sessions_full_mapped.csv")
#test_sessions= pd.concat([test_sessions, test_sessions_full])

In [691]:
test_sessions.head()

,session_id,item_id,date
0,61,20236,2021-06-01 08:12:53.513
1,96,19912,2021-06-19 18:00:35.142
2,185,13115,2021-06-07 15:55:53.129
3,224,23733,2021-06-14 16:04:34.465
4,285,20057,2021-06-29 15:35:04.948


In [692]:
test_sessions = test_sessions.drop(columns=["date"])

In [693]:
test_sessions.head()

,session_id,item_id
0,61,20236
1,96,19912
2,185,13115
3,224,23733
4,285,20057


In [694]:
final_comprasion_df = {}

for model_name in selected_list:
    final_comprasion_df[model_name] = pd.merge(test_sessions, submissions_df[model_name], on=["session_id", "item_id"], how="left").fillna(100)

In [695]:
final_comprasion_df["UCF_W"].head()

,session_id,item_id,rank
0,61,20236,2.0
1,96,19912,100.0
2,185,13115,4.0
3,224,23733,2.0
4,285,20057,2.0


In [696]:
mrr_dict = {}

In [697]:
for model_name in selected_list:
    mrr_dict[model_name] = (1 / final_comprasion_df[model_name].groupby('session_id')['rank'].min()).mean()

In [698]:
print(mrr_dict)

{'UCF_W': 0.19541221241438764, 'ICBF_W': 0.18029180839571288, 'Graph_W': 0.19380429297127796, 'ICF_W': 0.19046867373644116}


In [701]:
compression_table = pd.DataFrame([mrr_dict.values()], index=['MRR@100'], columns=[f"{x}+Transformer" for x in mrr_dict.keys()])

In [702]:
compression_table

,UCF_W+Transformer,ICBF_W+Transformer,Graph_W+Transformer,ICF_W+Transformer
MRR@100,0.195412,0.180292,0.193804,0.190469
